# Introduction

Roboquant is an open source algoritmic trading framework written in [Kotlin](https://kotlinlang.org). It enables fast and robust back-testing and live trading in a fully automatic way. You can read more about roboquant at [roboquant.org](https://roboquant.org)

As an algo-trader, you can use roboquant in two different ways:

1. In a Jupyter Notebook 
2. As a library in your standalone Kotlin/Java application using your IDE of choice

Both share exactly the same code-base, so there is no difference how to develop your strategies or how they behave. 

> The Notebook approach is a great to quickly be up and running and have a more iterative approach to developing new trading strategies. It comes also with charts that visualize how your strategy is performing.

> As your strategy grows, you might benefit from the power of a full-fledged IDE like IDEA and move to a standalone application. Also if you want to run it for live-trading, the standalone application might prove a more robust approach. 

This notebook demonstrates only the very basics of using roboquant and how to test a strategy. See the other included notebooks for a more advanced topics.

We start with the cell magic **%use roboquant** to load the required packages. The **Welcome()** statement is optional, but gives usefull info on the environment we are running in.

In [ ]:
%use roboquant
Welcome()

# Define a Roboquant
Besides the name of the software, the Roboquant class is also the engine of the framework. It ties all the components together that make-up a back-test. It also runs the actual tests and live trading.

There several default parameters for a roboquant, so the only two main ingredients remaining that need to be defined to get started are:

1. The strategy we want to test. For this example we use a Exponential Moving Average Crossover with its default settings. This comes out-of-the-box with roboquant.
2. Any metrics we want to captured during the run to see how our strategy is performing. In this case we use the AccountSummary metric.

roboquant allows for much more customization, but for this introduction notebook we'll keep it simple.  

In [ ]:
val strategy = EMACrossover()
val roboquant =  Roboquant(strategy, AccountSummary())

# Feed & Run
Now we define the data we want to use for the run. In roboqaunt data is provided by a Feed and there are many Feed providers included out of the box. They are ranging from Feeds for handling local CSV files to online data providers. In this notebook we use a the Avro Feed provider, which also comes with some S&P 500 test data. 

After the feed is created, we are all set to start the run. When you start a run, by default a progress bar is shown that provides visual feedback how the run is progressing. 

In [ ]:
val feed = AvroFeed.sp500()
roboquant.run(feed)

# Examine Results

Likely the first thing you want to know after the run has finished is: how does the account look like? 

An account in roboquant is owned by the boker and contains the following 4 types of data: 

1. the cash balances
2. the portfolio with its positions
3. the orders, both open and closed orders
4. any trades made, so orders that have been (partially) filled


> Please note the use of the **summary()** method in the next notebook cells. Many classes in roboquant implement this method and it provides a nice summary of the state on an object, like in this case the account and portfolio object.


In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
// You can also get more details, for example about the portfolio
account.portfolio.summary()

In [ ]:
val trades = account.trades
trades[0..10].summary()

# Metric Results
The account contains the state at the end of the run. Metrics are more powerful in that they capture state during each step of the run. So they give an overview how a metric evolves over time during the run. It is the logger who is responsible for storing or logging the metric results.  

In [ ]:
var logger = roboquant.logger
logger.getMetricNames()

In [ ]:
logger.summary(2)

In [ ]:
logger.getMetric("account.value").summary()

# More detailed inspections

After a quick scan of the results, you might want to dive into some more details. For example how much profit or loss did the strategy make on a single asset, like for example Apple?

In [ ]:
val apple = trades.assets.getBySymbol("AAPL")
trades.realizedPnL(apple)

Or how much profit did we make in each year?

In [ ]:
trades.timeline.split(1.years).forEach {
    val pnl = trades.realizedPnL(it)
    println("$it profit: $pnl")
}

# Charts
A great way to get this overview more easy is by using one of the many included charts. If you want to find out more about that, check out the visualization Notebook. 
In this notebook we just include an example that shows how to quickly spot which days had the best and worst performance during a back test.

In [ ]:
CalendarChart(logger.getMetric("account.change"))